<a href="https://colab.research.google.com/github/indra-code/GeniuneORFake_Reviews_Classifier/blob/main/GenuineORFake_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 #0 - Decision Tree
 #1 - Support Vector Machine
 #2 - Logistic Regression
 #3 - KNN
 #4 - Random Forest

 Not using Naive Bayes as mean of rating column is > 0 therefore resulting in negative values.


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
vectorizer = CountVectorizer()

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('processed_fake_reviews_dataset.csv')
df.head()

,Unnamed: 0,category,rating,label,text_
0,0,Home_and_Kitchen_5,5.0,CG,love well made sturdi comfort i love veri pretti
1,1,Home_and_Kitchen_5,5.0,CG,love great upgrad origin i 've mine coupl year
2,2,Home_and_Kitchen_5,5.0,CG,thi pillow save back i love look feel pillow
3,3,Home_and_Kitchen_5,1.0,CG,miss inform use great product price i
4,4,Home_and_Kitchen_5,5.0,CG,veri nice set good qualiti we set two month


In [26]:
from sklearn.pipeline import FeatureUnion, Pipeline,make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

text_pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer())
])
feature_pipeline = ColumnTransformer(
    transformers=[
        ('text', text_pipeline, 'text_'),
        ('scaler', StandardScaler(), ['rating']),
        ('encoder', OneHotEncoder(), ['category'])
    ]
)
full_pipeline = Pipeline([('data',feature_pipeline),('classifier',DecisionTreeClassifier())])

In [7]:
ColumnTransformer([LabelEncoder(),['label']])

ColumnTransformer(transformers=[LabelEncoder(), ['label']])

In [9]:
df.dropna(subset=['text_'], inplace=True)

In [31]:
le = LabelEncoder()
y = np.array(le.fit_transform(df['label']))
y
y.shape

(14832,)

In [11]:
X = df.iloc[:,[1,2,4]]
X

,category,rating,text_
0,Home_and_Kitchen_5,5.0,love well made sturdi comfort i love veri pretti
1,Home_and_Kitchen_5,5.0,love great upgrad origin i 've mine coupl year
2,Home_and_Kitchen_5,5.0,thi pillow save back i love look feel pillow
3,Home_and_Kitchen_5,1.0,miss inform use great product price i
4,Home_and_Kitchen_5,5.0,veri nice set good qualiti we set two month
...,...,...,...
14827,Movies_and_TV_5,2.0,i bought dvd buck i still overpaid i want like...
14828,Movies_and_TV_5,3.0,i love key peel admit movi silli i laugh aloud...
14829,Movies_and_TV_5,5.0,the follow film review rate pg-13 action seqeu...
14830,Movies_and_TV_5,5.0,thi movi encompass epidem world today there st...


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
X_train.shape

(11865, 3)

In [13]:
full_pipeline.fit(X_train,y_train)

Pipeline(steps=[('data',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('bow',
                                                                   CountVectorizer()),
                                                                  ('tfidf',
                                                                   TfidfTransformer())]),
                                                  'text_'),
                                                 ('scaler', StandardScaler(),
                                                  ['rating']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['category'])])),
                ('classifier', DecisionTreeClassifier())])

In [14]:
print(full_pipeline)

Pipeline(steps=[('data',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('bow',
                                                                   CountVectorizer()),
                                                                  ('tfidf',
                                                                   TfidfTransformer())]),
                                                  'text_'),
                                                 ('scaler', StandardScaler(),
                                                  ['rating']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['category'])])),
                ('classifier', DecisionTreeClassifier())])


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(X_train['text_'])
tfidf_matrix.shape

(11865, 13704)

In [16]:
X_train['text_'].shape

(11865,)

In [17]:
preds = full_pipeline.predict(X_test)
preds

array([0, 0, 1, ..., 0, 1, 0])

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print('Model Prediction Accuracy:',str(np.round(accuracy_score(y_test,preds)*100,2)) + '%')

Model Prediction Accuracy: 76.64%


In [20]:
from sklearn.svm import SVC
full_pipeline1 = Pipeline([('data',feature_pipeline),('classifier',SVC())])
full_pipeline1.fit(X_train,y_train)

Pipeline(steps=[('data',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('bow',
                                                                   CountVectorizer()),
                                                                  ('tfidf',
                                                                   TfidfTransformer())]),
                                                  'text_'),
                                                 ('scaler', StandardScaler(),
                                                  ['rating']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['category'])])),
                ('classifier', SVC())])

In [24]:
preds1 = full_pipeline1.predict(X_test)
preds1

array([1, 0, 1, ..., 1, 1, 0])

In [25]:
print('Classification Report:',classification_report(y_test,preds1))
print('Confusion Matrix:',confusion_matrix(y_test,preds1))
print('Accuracy Score:',accuracy_score(y_test,preds1))
print('Model Prediction Accuracy:',str(np.round(accuracy_score(y_test,preds1)*100,2)) + '%')

Classification Report:               precision    recall  f1-score   support

           0       0.92      0.84      0.87      1499
           1       0.85      0.92      0.88      1468

    accuracy                           0.88      2967
   macro avg       0.88      0.88      0.88      2967
weighted avg       0.88      0.88      0.88      2967

Confusion Matrix: [[1252  247]
 [ 115 1353]]
Accuracy Score: 0.8779912369396697
Model Prediction Accuracy: 87.8%


In [29]:
full_pipeline2 = Pipeline([('data',feature_pipeline),('classifier',LogisticRegression(max_iter = 1000))])
full_pipeline2.fit(X_train,y_train)

Pipeline(steps=[('data',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('bow',
                                                                   CountVectorizer()),
                                                                  ('tfidf',
                                                                   TfidfTransformer())]),
                                                  'text_'),
                                                 ('scaler', StandardScaler(),
                                                  ['rating']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['category'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [32]:
preds2 = full_pipeline2.predict(X_test)
preds2

array([1, 0, 1, ..., 1, 0, 0])

In [33]:
print('Classification Report:',classification_report(y_test,preds2))
print('Confusion Matrix:',confusion_matrix(y_test,preds2))
print('Accuracy Score:',accuracy_score(y_test,preds2))
print('Model Prediction Accuracy:',str(np.round(accuracy_score(y_test,preds2)*100,2)) + '%')

Classification Report:               precision    recall  f1-score   support

           0       0.89      0.86      0.88      1499
           1       0.86      0.90      0.88      1468

    accuracy                           0.88      2967
   macro avg       0.88      0.88      0.88      2967
weighted avg       0.88      0.88      0.88      2967

Confusion Matrix: [[1287  212]
 [ 152 1316]]
Accuracy Score: 0.8773171553758005
Model Prediction Accuracy: 87.73%


In [34]:
full_pipeline3 = Pipeline([('data',feature_pipeline),('classifier',KNeighborsClassifier())])
full_pipeline3.fit(X_train,y_train)

Pipeline(steps=[('data',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('bow',
                                                                   CountVectorizer()),
                                                                  ('tfidf',
                                                                   TfidfTransformer())]),
                                                  'text_'),
                                                 ('scaler', StandardScaler(),
                                                  ['rating']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['category'])])),
                ('classifier', KNeighborsClassifier())])

In [35]:
preds3 = full_pipeline3.predict(X_test)
preds3

array([0, 0, 0, ..., 1, 1, 0])

In [39]:
print('Classification Report:',classification_report(y_test,preds3))
print('Confusion Matrix:',confusion_matrix(y_test,preds3))
print('Accuracy Score:',accuracy_score(y_test,preds3))
print('Model Prediction Accuracy:',str(np.round(accuracy_score(y_test,preds3)*100,2)) + '%')

Classification Report:               precision    recall  f1-score   support

           0       0.59      0.92      0.72      1499
           1       0.81      0.34      0.48      1468

    accuracy                           0.63      2967
   macro avg       0.70      0.63      0.60      2967
weighted avg       0.70      0.63      0.60      2967

Confusion Matrix: [[1384  115]
 [ 973  495]]
Accuracy Score: 0.6332996292551398
Model Prediction Accuracy: 63.33%


In [38]:
full_pipeline4 = Pipeline([('data',feature_pipeline),('classifier',RandomForestClassifier())])
full_pipeline4.fit(X_train,y_train)

Pipeline(steps=[('data',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('bow',
                                                                   CountVectorizer()),
                                                                  ('tfidf',
                                                                   TfidfTransformer())]),
                                                  'text_'),
                                                 ('scaler', StandardScaler(),
                                                  ['rating']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['category'])])),
                ('classifier', RandomForestClassifier())])

In [41]:
preds4 = full_pipeline4.predict(X_test)
preds4

array([1, 0, 1, ..., 1, 0, 0])

In [42]:
print('Classification Report:',classification_report(y_test,preds4))
print('Confusion Matrix:',confusion_matrix(y_test,preds4))
print('Accuracy Score:',accuracy_score(y_test,preds4))
print('Model Prediction Accuracy:',str(np.round(accuracy_score(y_test,preds4)*100,2)) + '%')

Classification Report:               precision    recall  f1-score   support

           0       0.81      0.90      0.85      1499
           1       0.89      0.79      0.83      1468

    accuracy                           0.84      2967
   macro avg       0.85      0.84      0.84      2967
weighted avg       0.85      0.84      0.84      2967

Confusion Matrix: [[1354  145]
 [ 315 1153]]
Accuracy Score: 0.8449612403100775
Model Prediction Accuracy: 84.5%


In [54]:
testing = ColumnTransformer(transformers=[('scaler', StandardScaler(), ['rating'])])
t = testing.fit_transform(df[['rating']])
t

array([[0.6391699],
       [0.6391699],
       [0.6391699],
       ...,
       [0.6391699],
       [0.6391699],
       [0.6391699]])

In [55]:
print("Final report: \n")
print('Decision Tree Classifier:',str(np.round(accuracy_score(y_test,preds)*100,2)) + '%')
print('Support Vector Machine:',str(np.round(accuracy_score(y_test,preds1)*100,2)) + '%')
print('Logistic Regression:',str(np.round(accuracy_score(y_test,preds2)*100,2)) + '%')
print('KNN:',str(np.round(accuracy_score(y_test,preds3)*100,2)) + '%')
print('Random Forest',str(np.round(accuracy_score(y_test,preds4)*100,2)) + '%')

Final report: 

Decision Tree Classifier: 76.64%
Support Vector Machine: 87.8%
Logistic Regression: 87.73%
KNN: 63.33%
Random Forest 84.5%
